In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [12]:
data = pd.read_hdf('data/data.h5', 'data')
data['target'] = data['target'].fillna(0.)

In [13]:
train, valid = data.iloc[:-3801], data.iloc[-3801:]
len(train), len(valid)

(50000, 3801)

In [14]:
writer = tf.python_io.TFRecordWriter('train.tfrecord')

for index, row in train.iterrows():
    row = row.astype(np.float32)
    target = row['target']
    is_train = int(row['is_train'])
    del row['target']
    del row['is_train']
    row = np.log(row.values+1)

    example = tf.train.Example(features=tf.train.Features(feature={
        'index': bytes_feature(index.encode('utf8')),
        'values': bytes_feature(row.tostring()),
        'target': float_feature(target),
        'is_train': float_feature(is_train)
    }))
    writer.write(example.SerializeToString())
writer.close()

In [15]:
writer = tf.python_io.TFRecordWriter('valid.tfrecord')

for index, row in valid.iterrows():
    row = row.astype(np.float32)
    target = row['target']
    is_train = int(row['is_train'])
    del row['target']
    del row['is_train']
    row = np.log(row.values+1)

    example = tf.train.Example(features=tf.train.Features(feature={
        'index': bytes_feature(index.encode('utf8')),
        'values': bytes_feature(row.tostring()),
        'target': float_feature(target),
        'is_train': float_feature(is_train)
    }))
    writer.write(example.SerializeToString())
writer.close()